### Baseline model (Plus)에 있는 기능

- binary_load_dataset 메서드

데이터 비율 조정을 위한 파라미터 값을 기반으로 데이터를 로드하는 메서드로

주어진 binary_classification_data.csv 파일을 열고 데이터를 읽습니다. 데이터의 마지막 열에 종속 변수가 있으며, 해당 열의 값에 따라서 '1'인 pulsar 데이터와 '0'인 star 데이터를 각각 별도의 리스트에 저장합니다.

그 후, 실험에 사용될 데이터와 신경망의 입력 및 출력 크기를 전역 변수로 선언합니다. pulsar와 star 데이터는 하나의 변수 data에 그룹화됩니다. pulsar 데이터의 수를 star 데이터의 수에 맞추기 위해 데이터의 수를 조정할 수 있습니다.

adjust_ratio 매개변수가 True인 경우, pulsar 데이터의 비율을 증가시킵니다. pulsar 데이터와 star 데이터를 하나의 변수에 할당하고, 데이터의 크기는 star 데이터의 배수로 저장됩니다.

adjust_ratio 값이 True인 경우 star 데이터를 data 변수에 할당한 후, pulsar 데이터를 star 데이터의 수만큼 반복하여 추가합니다. 이를 위해 % 연산자를 사용하여 pulsar 데이터의 인덱스를 순차적으로 추출합니다.

adjust_ratio 값이 False인 경우, 기존 데이터를 그대로 data 변수에 할당합니다. star 데이터를 먼저 할당하고, 그 다음에 pulsar 데이터를 할당합니다.

### 데이터 불러오기

In [1]:
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
from keras.layers import Dropout
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

### 데이터 불균형 처리
- 데이터 처리

1. oversampling :

   SMOTE, ADASYN

2. undersampling

- 평가지표 처리

1. accurcacy :

   대부분의 예측이 다수 클래스에 속하게 되어 별로

2. F1-score :

   Baseline model에서 사용한 평가지표로 불균형한 데이터에서 평가지표로 사용

- 모델 구현시 처리

1. 클래스 가중치 설정 :

   소수 클래스에 더 큰 가중치 부여


In [2]:
# 새로운 데이터 (EDA)과정 진행 후 데이터
X_train_smote = pd.read_csv('csv/Binary_to_share/Binary_to_share/Normal+Standard/X_train_NS_smote.csv')
X_train_adasyn = pd.read_csv('csv/Binary_to_share/Binary_to_share/Normal+Standard/X_train_NS_adasyn.csv')
X_test = pd.read_csv('csv/Binary_to_share/Binary_to_share/Normal+Standard/X_test_NS.csv')
X_val = pd.read_csv('csv/Binary_to_share/Binary_to_share/Normal+Standard/X_val_NS.csv')
y_train_smote = pd.read_csv('csv/Binary_to_share/Binary_to_share/Normal+Standard/y_train_NS_smote.csv')
y_train_adasyn = pd.read_csv('csv/Binary_to_share/Binary_to_share/Normal+Standard/y_train_NS_adasyn.csv')
y_test = pd.read_csv('csv/Binary_to_share/Binary_to_share/Normal+Standard/y_test.csv')
y_val = pd.read_csv('csv/Binary_to_share/Binary_to_share/Normal+Standard/y_val.csv')

In [3]:
# 클래스 가중치 계산
class_weight1 = {0: 1., 1: (len(y_train_smote) / y_train_smote['target_class'].sum())}
class_weight2 = {0: 1., 1: (len(y_train_adasyn) / y_train_adasyn['target_class'].sum())}

### 모델 구현 
- 최적의 파라미터를 반복문을 통해 찾기

- 이진 분류 모델에서는 대표적으로 Adam optimizer를 사용하는 것이 일반적입니다. 다른 optimizer로는 SGD, RMSprop 등이 있으며, 이들 중에서도 데이터셋의 크기와 모델의 복잡도에 따라 적합한 optimizer를 선택해야 합니다. 하지만 대체로 Adam optimizer가 다른 optimizer보다 성능이 좋은 편입니다.

- 모델에 배치 정규화 레이어를 추가하면 모델이 더 안정적으로 수렴할 수 있습니다. 

In [5]:
# 복습한 내용으로 추가적인 진행 - adam 사용, adasyn
def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt


# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
batch_lst =[32, 64, 128, 256]
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}

train_lst = [X_train_adasyn, X_train_smote]

best_metric = 0.0
metric_hyper = {}


# dropout, 배치 정규화 추가
# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            metrics_lst = []

            # model구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_shape=(X_train_adasyn.shape[1],)))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(64, activation=func))
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(1, activation=func))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
           
            model.fit(X_train_adasyn, y_train_adasyn, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], class_weight=class_weight2)
            tmp = class_weight2
            

            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}


            # 모델 평가
            y_pred = model.predict(X_val)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            metrics_lst.append(f1_score(y_val, y_pred_binary))
            metrics_lst.append(precision_score(y_val, y_pred_binary))
            metrics_lst.append(recall_score(y_val, y_pred_binary))
            mean_met = np.mean(metrics_lst)

            if mean_met > best_metric:
                best_metric = mean_met
                metric_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print('Best time:', best_time)
print('Best time_hyperparams:', time_hyper)
print('Best metric:', best_metric)
print('Best metric_hyperparams:', metric_hyper)


Epoch 1/1000
616/616 [==============================] - 7s 5ms/step - loss: 1.2629 - accuracy: 0.7485 - val_loss: 0.4679 - val_accuracy: 0.7798
Epoch 2/1000
616/616 [==============================] - 3s 4ms/step - loss: 0.8168 - accuracy: 0.7083 - val_loss: 0.6793 - val_accuracy: 0.8537
Epoch 3/1000
616/616 [==============================] - 3s 4ms/step - loss: 0.7274 - accuracy: 0.7113 - val_loss: 0.5926 - val_accuracy: 0.8224
Epoch 4/1000
616/616 [==============================] - 3s 4ms/step - loss: 0.6698 - accuracy: 0.7830 - val_loss: 0.6513 - val_accuracy: 0.7316
Epoch 5/1000
616/616 [==============================] - 3s 4ms/step - loss: 0.7410 - accuracy: 0.7676 - val_loss: 0.9723 - val_accuracy: 0.5673
Epoch 6/1000
616/616 [==============================] - 3s 4ms/step - loss: 0.6791 - accuracy: 0.7770 - val_loss: 0.5684 - val_accuracy: 0.7746
Epoch 7/1000
616/616 [==============================] - 3s 5ms/step - loss: 0.6041 - accuracy: 0.7952 - val_loss: 0.6124 - val_accuracy:

c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 206ms/epoch - 2ms/step
Epoch 1/1000
77/77 [==============================] - 7s 16ms/step - loss: 0.9608 - accuracy: 0.7400 - val_loss: 0.4657 - val_accuracy: 0.9335
Epoch 2/1000
77/77 [==============================] - 1s 7ms/step - loss: 0.9077 - accuracy: 0.7370 - val_loss: 0.9275 - val_accuracy: 0.7908
Epoch 3/1000
77/77 [==============================] - 1s 8ms/step - loss: 0.7174 - accuracy: 0.7868 - val_loss: 0.5625 - val_accuracy: 0.8408
Epoch 4/1000
77/77 [==============================] - 0s 6ms/step - loss: 1.4718 - accuracy: 0.6283 - val_loss: 1.9622 - val_accuracy: 0.0960
Epoch 5/1000
77/77 [==============================] - 0s 6ms/step - loss: 0.8024 - accuracy: 0.7236 - val_loss: 1.2015 - val_accuracy: 0.7085
Epoch 6/1000
77/77 [==============================] - 1s 8ms/step - loss: 0.7538 - accuracy: 0.7520 - val_loss: 0.7384 - val_accuracy: 0.6430
Epoch 7/1000
77/77 [==============================] - 1s 8ms/step - loss: 0.6

c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 186ms/epoch - 2ms/step
Epoch 1/1000
616/616 [==============================] - 8s 6ms/step - loss: 0.8369 - accuracy: 0.7737 - val_loss: 0.5850 - val_accuracy: 0.8213
Epoch 2/1000
616/616 [==============================] - 3s 5ms/step - loss: 0.7110 - accuracy: 0.7971 - val_loss: 0.3946 - val_accuracy: 0.8721
Epoch 3/1000
616/616 [==============================] - 3s 5ms/step - loss: 0.6671 - accuracy: 0.7994 - val_loss: 1.8159 - val_accuracy: 0.6798
Epoch 4/1000
616/616 [==============================] - 3s 5ms/step - loss: 0.8046 - accuracy: 0.7391 - val_loss: 0.6350 - val_accuracy: 0.7618
Epoch 5/1000
616/616 [==============================] - 3s 5ms/step - loss: 0.6918 - accuracy: 0.7879 - val_loss: 0.6732 - val_accuracy: 0.7327
Epoch 6/1000
616/616 [==============================] - 3s 6ms/step - loss: 0.6558 - accuracy: 0.8011 - val_loss: 0.6085 - val_accuracy: 0.7684
Epoch 7/1000
616/616 [==============================] - 3s 6ms/ste

c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 201ms/epoch - 2ms/step
Epoch 1/1000
308/308 [==============================] - 8s 9ms/step - loss: 5.6806 - accuracy: 0.6345 - val_loss: 6.3216 - val_accuracy: 0.5772
Epoch 2/1000
308/308 [==============================] - 2s 8ms/step - loss: 7.3446 - accuracy: 0.5246 - val_loss: 13.7860 - val_accuracy: 0.0960
Epoch 3/1000
308/308 [==============================] - 3s 8ms/step - loss: 7.6149 - accuracy: 0.5006 - val_loss: 13.7860 - val_accuracy: 0.0960
Epoch 4/1000
308/308 [==============================] - 3s 8ms/step - loss: 7.6149 - accuracy: 0.5006 - val_loss: 13.7860 - val_accuracy: 0.0960
Epoch 5/1000
308/308 [==============================] - 2s 8ms/step - loss: 7.6149 - accuracy: 0.5006 - val_loss: 13.7860 - val_accuracy: 0.0960
Epoch 6/1000
308/308 [==============================] - 3s 8ms/step - loss: 7.6149 - accuracy: 0.5006 - val_loss: 13.7860 - val_accuracy: 0.0960
Epoch 7/1000
308/308 [==============================] - 3s 8m

In [6]:
# 복습한 내용으로 추가적인 진행 - adam 사용, smote
def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt


# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
batch_lst =[32, 64, 128, 256]
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}


best_metric = 0.0
metric_hyper = {}


# dropout, 배치 정규화 추가
# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            metrics_lst = []

            # model구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_shape=(X_train_smote.shape[1],)))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(64, activation=func))
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(1, activation=func))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

        
            model.fit(X_train_smote, y_train_smote, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], class_weight=class_weight1)
            tmp = class_weight1
         


            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}


            # 모델 평가
            y_pred = model.predict(X_val)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            metrics_lst.append(f1_score(y_val, y_pred_binary))
            metrics_lst.append(precision_score(y_val, y_pred_binary))
            metrics_lst.append(recall_score(y_val, y_pred_binary))
            mean_met = np.mean(metrics_lst)

            if mean_met > best_metric:
                best_metric = mean_met
                metric_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print('Best time:', best_time)
print('Best time_hyperparams:', time_hyper)
print('Best metric:', best_metric)
print('Best metric_hyperparams:', metric_hyper)


Epoch 1/1000
615/615 [==============================] - 6s 4ms/step - loss: 0.4928 - accuracy: 0.9171 - val_loss: 0.3569 - val_accuracy: 0.9636
Epoch 2/1000
615/615 [==============================] - 2s 4ms/step - loss: 0.6250 - accuracy: 0.9001 - val_loss: 0.3119 - val_accuracy: 0.9710
Epoch 3/1000
615/615 [==============================] - 2s 4ms/step - loss: 0.5577 - accuracy: 0.9230 - val_loss: 0.5280 - val_accuracy: 0.9702
Epoch 4/1000
615/615 [==============================] - 3s 4ms/step - loss: 0.5879 - accuracy: 0.9149 - val_loss: 0.6161 - val_accuracy: 0.9577
Epoch 5/1000
615/615 [==============================] - 3s 4ms/step - loss: 0.6099 - accuracy: 0.9168 - val_loss: 0.9494 - val_accuracy: 0.9390
Epoch 6/1000
615/615 [==============================] - 3s 4ms/step - loss: 1.1862 - accuracy: 0.8839 - val_loss: 1.6215 - val_accuracy: 0.9004
Epoch 7/1000
615/615 [==============================] - 3s 4ms/step - loss: 1.1155 - accuracy: 0.8815 - val_loss: 1.5896 - val_accuracy:

c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 162ms/epoch - 2ms/step
Epoch 1/1000
308/308 [==============================] - 4s 6ms/step - loss: 15.4249 - accuracy: 0.5000 - val_loss: 1.4801 - val_accuracy: 0.9040
Epoch 2/1000
308/308 [==============================] - 1s 5ms/step - loss: 15.4250 - accuracy: 0.5000 - val_loss: 1.4801 - val_accuracy: 0.9040
Epoch 3/1000
308/308 [==============================] - 1s 5ms/step - loss: 15.4250 - accuracy: 0.5000 - val_loss: 1.4801 - val_accuracy: 0.9040
Epoch 4/1000
308/308 [==============================] - 2s 5ms/step - loss: 15.4249 - accuracy: 0.5000 - val_loss: 1.4801 - val_accuracy: 0.9040
Epoch 5/1000
308/308 [==============================] - 1s 5ms/step - loss: 15.4249 - accuracy: 0.5000 - val_loss: 1.4801 - val_accuracy: 0.9040
Epoch 6/1000
308/308 [==============================] - 1s 5ms/step - loss: 15.4250 - accuracy: 0.5000 - val_loss: 1.4801 - val_accuracy: 0.9040
Epoch 7/1000
308/308 [==============================] - 1s 5

c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 174ms/epoch - 2ms/step
Epoch 1/1000
154/154 [==============================] - 4s 8ms/step - loss: 0.4248 - accuracy: 0.9083 - val_loss: 0.2164 - val_accuracy: 0.9588
Epoch 2/1000
154/154 [==============================] - 1s 6ms/step - loss: 0.3352 - accuracy: 0.9237 - val_loss: 0.2505 - val_accuracy: 0.9636
Epoch 3/1000
154/154 [==============================] - 1s 6ms/step - loss: 0.3369 - accuracy: 0.9290 - val_loss: 0.2290 - val_accuracy: 0.9603
Epoch 4/1000
154/154 [==============================] - 1s 6ms/step - loss: 0.3884 - accuracy: 0.9192 - val_loss: 0.2851 - val_accuracy: 0.9529
Epoch 5/1000
154/154 [==============================] - 1s 6ms/step - loss: 0.3353 - accuracy: 0.9247 - val_loss: 0.2360 - val_accuracy: 0.9592
Epoch 6/1000
154/154 [==============================] - 1s 6ms/step - loss: 0.4373 - accuracy: 0.9045 - val_loss: 0.3569 - val_accuracy: 0.9393
Epoch 7/1000
154/154 [==============================] - 1s 6ms/ste

c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 172ms/epoch - 2ms/step
Epoch 1/1000
154/154 [==============================] - 4s 7ms/step - loss: 1.7049 - accuracy: 0.7074 - val_loss: 0.1528 - val_accuracy: 0.9665
Epoch 2/1000
154/154 [==============================] - 1s 5ms/step - loss: 0.3801 - accuracy: 0.9207 - val_loss: 0.1685 - val_accuracy: 0.9658
Epoch 3/1000
154/154 [==============================] - 1s 6ms/step - loss: 0.3734 - accuracy: 0.9247 - val_loss: 0.2423 - val_accuracy: 0.9559
Epoch 4/1000
154/154 [==============================] - 1s 6ms/step - loss: 0.3490 - accuracy: 0.9282 - val_loss: 0.3113 - val_accuracy: 0.9485
Epoch 5/1000
154/154 [==============================] - 1s 5ms/step - loss: 0.3493 - accuracy: 0.9305 - val_loss: 0.2579 - val_accuracy: 0.9621
Epoch 6/1000
154/154 [==============================] - 1s 5ms/step - loss: 0.3266 - accuracy: 0.9249 - val_loss: 0.2441 - val_accuracy: 0.9555
Epoch 7/1000
154/154 [==============================] - 1s 5ms/ste